# TP2 : Affectation

In [5]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;

Dans la commune de *Rolette*, il y a 3 quartiers et 3 maternelles. 
Les coûts de transport par élève des 3 zones aux différentes écoles sont donnés dans le tableau ci-dessous. 
Le signe `-` indique une affectation impossible.

Zone | Nombre d'élèves | École 1 | École 2 | École 3
-----|-----------------|---------|--------|----------
1 | 450 | 300 | - | 700
2 | 600 | -  | 400| 500
3 | 550 | 600 | 300 | 200



Voici la capacité d'acceuil de chaque école :

 École 1 | École 2 | École 3
---------|--------|----------
700 | 450 | 600

## Exercice 1

Quelle est l'affectation des élèves dans les trois écoles, qui minimise le coût global de transport pris en charge par la commune ?

Modéliser et résoudre ce problème en JuMP.

Pour cela, on utilisera une variable entière pour chaque zone et chaque école (dont l'affectation est possible) indiquant le nombre d'élèves de la zone affectés à l'école.

Les contraintes sont :

* contraintes d'affectation : tous les élèves doivent être afféctés à une école
* contraintes de capacité : une école ne peut pas accueillir plus d'élève que sa capacité


In [4]:
############################## 
#   Saisir votre code ici.   #
##############################
function optimise_affectation()

    affectation = Model(GLPK.Optimizer)

    #variables xij entières : combien d'élèves de la zone i sont affectés à l'école j
    @variable(affectation, 0 <= x11, Int)
    @variable(affectation, 0 <= x13, Int)
    @variable(affectation, 0 <= x22, Int)
    @variable(affectation, 0 <= x23, Int)
    @variable(affectation, 0 <= x31, Int)
    @variable(affectation, 0 <= x32, Int)
    @variable(affectation, 0 <= x33, Int)

    #La fonction objectif : On cherche à minimiser le nombre de rouleaux decoupes.
    @objective(affectation, Min, 300x11 + 700x13 + 400x22 + 500x23 + 600x31 + 300x32 + 200x33)

    #contrainte affectation par zone
    @constraint(affectation, x11 + x13 == 450)
    @constraint(affectation, x22 + x23 == 600)
    @constraint(affectation, x31 + x32 + x33 == 550)


    #contrainte capacité
    @constraint(affectation, x11 + x31 <= 700)
    @constraint(affectation, x22 + x32 <= 450)
    @constraint(affectation, x13 + x23 + x33 <= 600)


    JuMP.optimize!(affectation)

    println("le coût minimum de transport est : ", objective_value(affectation))

    #Affichage du résultat

    println("Nombre d'élèves de la zone 1 à l'école 1 = ", JuMP.value(x11))
    println("Nombre d'élèves de la zone 1 à l'école 3 = ", JuMP.value(x13))
    println("Nombre d'élèves de la zone 2 à l'école 2 = ", JuMP.value(x22))
    println("Nombre d'élèves de la zone 2 à l'école 3 = ", JuMP.value(x23))
    println("Nombre d'élèves de la zone 3 à l'école 1 = ", JuMP.value(x31))
    println("Nombre d'élèves de la zone 3 à l'école 2 = ", JuMP.value(x32))
    println("Nombre d'élèves de la zone 3 à l'école 3 = ", JuMP.value(x33))

end

optimise_affectation()

le coût minimum de transport est : 540000.0
Nombre d'élèves de la zone 1 à l'école 1 = 450.0
Nombre d'élèves de la zone 1 à l'école 3 = 0.0
Nombre d'élèves de la zone 2 à l'école 2 = 450.0
Nombre d'élèves de la zone 2 à l'école 3 = 150.0
Nombre d'élèves de la zone 3 à l'école 1 = 100.0
Nombre d'élèves de la zone 3 à l'école 2 = 0.0
Nombre d'élèves de la zone 3 à l'école 3 = 450.0


## Exercice 2

### Question 1

Écrire sur feuille une formulation générique du problème d'affectation de coût minimum, c'est-à-dire une formulation qui dépend des paramètres suivants :

* $E$ : l'ensemble des écoles,
* $Z$ : l'ensemble des zones,
* $P_j$ : capacité de l'école $j$,
* $n_i$ : nombre d'élèves de la zone $i$,
* $c_{i,j}$ : coût de transport d'un élève de la zone $i$ vers l'école $j$.

Pour cela, on utilisera les variables $x_{i,j}$ correspondant au nombre d'enfants de la zone $i$ affectés à l'école $j$.

### Question 2

Écrire cette formulation générique en JuMP. Pour cela, on utilisera des variables indexées par des ensembles. Vous trouverez la documentation et les exemples avec les liens suivants.

* [Documentation](http://www.juliaopt.org/JuMP.jl/v0.20.0/)
* [Exemples](https://github.com/JuliaOpt/JuMP.jl/tree/release-0.20/examples)

Résoudre le problème donné par les valeurs suivantes : 

In [2]:
# Coûts d'affectation des zones aux écoles (0 signifie affectation impossible)
c = [300 0 700;
    0 400 500;
    600 300 200;
    200 500 0;
    0 0 400;
    500 300 0]

# Capacité des écoles
P = [900 1100 1000]

# Nombre d'élèves dans chaque zone
n = [450 600 550 350 500 450]

1×6 Array{Int64,2}:
 450  600  550  350  500  450

In [3]:
############################## 
#   Saisir votre code ici.   #
##############################

In [ ]:
function exo2_q2()
    # create a model
    m = Model(GLPK.Optimizer)
    
    # create variable
    @variable(m, c11 >= 0, Int)
    @variable(m, c13 >= 0, Int)
    @variable(m, c22 >= 0, Int)
    @variable(m, c23 >= 0, Int)
    @variable(m, c31 >= 0, Int)
    @variable(m, c32 >= 0, Int)
    @variable(m, c33 >= 0, Int)
    @variable(m, c41 >= 0, Int)
    @variable(m, c42 >= 0, Int)
    @variable(m, c53 >= 0, Int)
    @variable(m, c61 >= 0, Int)
    @variable(m, c62 >= 0, Int)
    
    # la fonction objecti
    @objective(m, Min, 300c11+700c13+400c22+500c23+600c31+300c32+200c33+200c41+500c42+400c53+500c61+300c62)
end

In [7]:
z = 5
for i in 1:z
    println(i)
end
arr = [1,2,3,4,5]


1
2
3
4
5


LoadError: syntax: incomplete: premature end of input